Beginning of a skeleton NN notebook: try to match the architecture provided in the papers.

In [115]:
# Import libraries
import matplotlib 
from matplotlib import pyplot as plt

import keras
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Set larger fontsize for all plots
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Setting up the data will be simple:

In [116]:
# Load the data
df = pd.read_csv('mol_res_scan_results_7.csv')
df.head()

,Unnamed: 0,cut 1,cut 2,yield,purity,c_s,v,mol1_D_L,mol1_k_ov,mol1_q_max,...,mol1_z_p,mol1_n,mol1_K_s,mol2_D_L,mol2_k_ov,mol2_q_max,mol2_K_eq,mol2_z_p,mol2_n,mol2_K_s
0,0,0.0,3.0,0.242987,0.601903,0.2,0.576042,0.059,0.542,38.723139,...,2.79852,0,0,0.059,0.542,24.871645,13.137022,2.49598,0,0
1,1,0.0,6.0,0.340549,0.610753,0.2,0.576042,0.059,0.542,38.723139,...,2.79852,0,0,0.059,0.542,24.871645,13.137022,2.49598,0,0
2,2,0.0,9.0,0.401249,0.629828,0.2,0.576042,0.059,0.542,38.723139,...,2.79852,0,0,0.059,0.542,24.871645,13.137022,2.49598,0,0
3,3,0.0,12.0,0.450786,0.647608,0.2,0.576042,0.059,0.542,38.723139,...,2.79852,0,0,0.059,0.542,24.871645,13.137022,2.49598,0,0
4,4,0.0,15.0,0.493797,0.663042,0.2,0.576042,0.059,0.542,38.723139,...,2.79852,0,0,0.059,0.542,24.871645,13.137022,2.49598,0,0


In [117]:
seed = 42 

# do we need to get rid of cut 1/ cut 2? what are the important features here?

X = df[['cut 1','cut 2','c_s','v','mol1_D_L','mol1_k_ov','mol1_q_max','mol1_K_eq','mol1_z_p','mol1_n','mol1_K_s','mol2_D_L','mol2_k_ov','mol2_q_max','mol2_K_eq','mol2_z_p','mol2_n','mol2_K_s']].values
Y = df[['yield','purity']].values

# Split into testing and training
X_train_pn, X_test_pn, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=seed)

In [118]:
# Scaling
X_train_scaler = StandardScaler().fit(X_train_pn)
X_train = X_train_scaler.transform(X_train_pn)
X_test = X_train_scaler.transform(X_test_pn)

The following needs to be modified to suit our needs:

    Model: sequential or API?
    Initializer: currently pulling from a normal distribution; is this necessary to change?
    Activation function: Relu v. sigmoid?
    Optimizer: Using Adam; is this necessary to change?
    Number of neurons
    Number of layers
    Learning rate?
    
Nagrath

    Sigmoidal Transfer Function
    Two hidden layers with 10 and 7 neurons, respectively.
    Output layer transfer function is linear.

In [135]:
# Create the neural network model

def simple_nn():

    model = Sequential()
    model.add(Dense(10, input_dim=18, kernel_initializer='normal', activation='sigmoid'))
    model.add(Dropout(0.2))
    model.add(Dense(2, kernel_initializer='normal'))
    opt = keras.optimizers.Adam(learning_rate=0.5)
    
    model.compile(loss='mean_squared_error', optimizer=opt)
    
    return model

In [136]:
# Train the model
estimator = KerasRegressor(build_fn=simple_nn, epochs=150, batch_size=100, verbose=0)
history = estimator.fit(X_train, y_train, validation_split=0.30, epochs=150, batch_size=100, verbose=0)
print("Final MSE for train is %.3e and for validation is %.3e" % 
      (history.history['loss'][-1], history.history['val_loss'][-1]))

Final MSE for train is 1.670e-02 and for validation is 1.080e-02


In [137]:
#history.history

In [138]:
# Evaluating model
test_loss = estimator.model.evaluate(X_test, y_test)
print("test set mse is %.3e" % test_loss)

3300/3300 [==============================] - 0s 21us/step
test set mse is 1.102e-02


In [139]:
predictions = estimator.model.predict(X_test)

In [140]:
X_test

array([[-8.12546133e-01,  8.13012127e-01,  1.00000000e+00, ...,
        -1.01617147e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.22039498e+00, -4.07847159e-01,  1.00000000e+00, ...,
        -1.06328580e+00,  0.00000000e+00,  0.00000000e+00],
       [-8.12546133e-01, -8.14800254e-01,  1.00000000e+00, ...,
        -7.43182984e-01,  0.00000000e+00,  0.00000000e+00],
       ...,
       [-8.12546133e-01, -8.94063619e-04,  1.00000000e+00, ...,
        -1.06512772e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.22039498e+00, -1.62870644e+00,  1.00000000e+00, ...,
         1.51002682e+00,  0.00000000e+00,  0.00000000e+00],
       [-8.12546133e-01,  8.13012127e-01,  1.00000000e+00, ...,
         1.39101376e+00,  0.00000000e+00,  0.00000000e+00]])

In [141]:
y_test

array([[0.44813997, 0.79035513],
       [0.65746378, 0.72971574],
       [0.6252425 , 0.81927121],
       ...,
       [0.54559589, 0.85612038],
       [0.21303454, 0.75483561],
       [0.79267044, 0.83447293]])

In [142]:
predictions

array([[0.5412758 , 0.8060433 ],
       [0.54264975, 0.8061149 ],
       [0.5382356 , 0.80588484],
       ...,
       [0.54283077, 0.8061244 ],
       [0.23842373, 0.7902551 ],
       [0.54297984, 0.80613214]], dtype=float32)